<a href="https://colab.research.google.com/github/duckietown/duckietown-lx-recipes/blob/mooc2022/object-detection/assets/colab/dt_object_detection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [3]:
from google.colab import drive
drive.mount('/content/drive')

with directory("/content/drive/My Drive/duckietown_object_detection_dataset"):
  print(os.getcwd())
  print(os.listdir())

os.chdir('/content/drive/My Drive/duckietown_object_detection_dataset')


Mounted at /content/drive
/content/drive/My Drive/duckietown_object_detection_dataset
['train', 'frames', 'labels', 'annotation', 'val', 'images']


In [4]:

if not os.path.exists("SENTINEL"):
  prun("mkdir duckietown_object_detection_dataset")
  prun("mv train duckietown_object_detection_dataset && mv val duckietown_object_detection_dataset")

None
None


# Next, we will clone Yolov5

In [5]:
!git clone https://github.com/duckietown/yolov5.git -b dt-obj-det
!cd yolov5 && pip3 install -r requirements.txt
!pip3 install torch==1.11 torchvision==0.12.0
if not os.path.exists("SENTINEL"):
  run("mv duckietown_object_detection_dataset yolov5")
!touch SENTINEL

Cloning into 'yolov5'...
remote: Enumerating objects: 6162, done.
remote: Total 6162 (delta 0), reused 0 (delta 0), pack-reused 6162
Receiving objects: 100% (6162/6162), 8.47 MiB | 11.72 MiB/s, done.
Resolving deltas: 100% (4212/4212), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 750.6 MB 18 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the pack

# We can now inform the training process of the format and location of our dataset

In [8]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../train
val: ../val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Overwriting ./yolov5/data/duckietown.yaml


# And we're ready to train! This step will take a hour or so.

Notice that we're only training for 10 epochs. That's probably not enough!

In [10]:
!mv yolov5/best.pt yolov5/best_old.pt
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 10 --data duckietown.yaml --weights yolov5s.pt

mv: cannot stat 'yolov5/best.pt': No such file or directory
github: up to date with https://github.com/duckietown/yolov5 ✅
YOLOv5 🚀 v5.0-68-ge6681ef torch 1.11.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/duckietown.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp3', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937

In [11]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")

Latest exp is exp3
None


# Next, we can upload your model to Duckietown's cloud!

In [12]:
!pip3 install git+https://github.com/duckietown/lib-dt-mooc-2021

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/duckietown/lib-dt-mooc-2021 to /tmp/pip-req-build-fcddid85
  Running command git clone -q https://github.com/duckietown/lib-dt-mooc-2021 /tmp/pip-req-build-fcddid85
     |████████████████████████████████| 142 kB 40.8 MB/s 
  Created wheel for dt-mooc-2021: filename=dt_mooc_2021-0.0.0-py3-none-any.whl size=6447 sha256=c5a28454b046516c1a2b35bcb7225ad0d9e55167caf5f8bfad62a7ac4385603b
  Stored in directory: /tmp/pip-ephem-wheel-cache-pjw9lrp7/wheels/d5/df/21/aa957c23320a8241d9197289a987390c37adce3463c5ef9df8
  Created wheel for dt-data-api-daffy: filename=dt_data_api_daffy-1.1.3-py3-none-any.whl size=13714 sha256=1a15ce669a72ef8ea7c8333a18f1eaf3e15b3dd873df59c43946b88ced02f8a3
  Stored in directory: /root/.cache/pip/wheels/38/ce/fb/5a988e5766d210a7586879219893e14a85ebca9f179ee91041
  Created wheel for dt-authentication-daffy: filename=dt_authentication_daffy-0.1.1

In [13]:
from dt_mooc.cloud import Storage
import sys
import torch

def select_device(device='', batch_size=None):
    import torch
    # device = 'cpu' or '0' or '0,1,2,3'
    cpu = device.lower() == 'cpu'
    if cpu:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # force torch.cuda.is_available() = False
    elif device:  # non-cpu device requested
        os.environ['CUDA_VISIBLE_DEVICES'] = device  # set environment variable
        assert torch.cuda.is_available(), f'CUDA unavailable, invalid device {device} requested'  # check availability

    cuda = not cpu and torch.cuda.is_available()

    return torch.device('cuda:0' if cuda else 'cpu')

sys.path.insert(0, './yolov5')
model = torch.load("./yolov5/best.pt", map_location=select_device("cpu"))['model'].float()  # load to FP32
model.to(select_device("cpu")).eval()

storage = Storage("YOUR_DUCKIETOWN_TOKEN_HERE")

storage.upload_yolov5("yolov5", model, "./yolov5/best.pt")

Uploading file `best.pt`...

File `best.pt` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.pt`.
Uploading file `best.pt.wts`...

File `best.pt.wts` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.wts`.
Uploading file `best.pt.wts.sha256`...

File `best.pt.wts.sha256` successfully uploaded! It will now be found at `courses/mooc/2021/data/nn_models/yolov5.sha256`.


# Done!

We're done training! You can go back to the `Training` notebook